# Hyperparameter Optimization with RandomSearchCV (BONUS)

RandomSearchCV, verilecek bir parametre seti içerisinden rastgele seçimler yapar ve bu rastgele seçimleri arar. Örnek olarak bu işlemi bir random forest modeli için gerçekleştirelim.

In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

warnings.simplefilter(action='ignore', category=Warning)

# from xgboost import XGBClassifier
# !pip install lightgbm
# from lightgbm import LGBMClassifier
# !pip install catboost
# from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv("datasets/diabetes.csv")

In [3]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [4]:
rf_model = RandomForestClassifier(random_state=17)

In [5]:
rf_random_params = {"max_depth": np.random.randint(5, 50, 10),
                   "max_features": [3, 5, 7, "auto", "sqrt"],
                   "min_samples_split": np.random.randint(2, 50, 20),
                   "n_estimators": [int(x) for x in np.linspace(start=200, stop=1500, num=10)]}

RandomSearch yöntemi, GridSearch yöntemine göre daha fazla daha geniş bir hiperparametre adayı arasından rastgele seçim yapar, bu seçtikleri üzerinden tek tek deneme yapar. Örneğin 5000 tane olası kombinasyondan 100 tanesini rastgele seç, bu seçtiğin 100 tanesi üzerinde aramaları yap.

Gidilecek yön bilinmiyorsa, ilgili algoritma üzerinde daha önce detaylı bir çalışma imkanı bulunmadıysa, önce bir RandomSearchCV'i getirip, buraya çok geniş bir hiperparametre seti girip, önce randomsearch ile burada arama yapıp, daha sonra burada randomsearch'e göre optimum olarak bulunan hiperparametrelerin etrafına yeni değerler koyarak daha az sayıda yeni değer koyarak GridSearchCV yönteminden geçirilebilir.

In [6]:
rf_random = RandomizedSearchCV(estimator=rf_model,
                              param_distributions=rf_random_params,
                              n_iter=100, #denenecek parametre sayısı
                              cv=3,
                              verbose=True,
                              random_state=42,
                              n_jobs=-1).fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  9.0min finished


In [7]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 13,
 'max_features': 'auto',
 'max_depth': 45}

In [9]:
rf_final = rf_model.set_params(**rf_random.best_params_, random_state=17).fit(X, y)

In [11]:
cv_results = cross_validate(rf_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print("Accuracy: ", cv_results["test_accuracy"].mean())
print("F1: ", cv_results["test_f1"].mean())
print("ROC-AUC: ", cv_results["test_roc_auc"].mean())

Accuracy:  0.7644087938205585
F1:  0.6330145289426738
ROC-AUC:  0.8349091544374563
